In [155]:
import requests
import pandas as pd
import numpy as np
import re
import pygsheets

In [156]:
BASE_URL = "https://www.openpowerlifting.org/u/" 
cols = ['Last Name','First Name','openpowerlifting link', 'instagram', 'raw squat','raw bench','raw DL','raw total', 'eq squat','eq bench','eq DL','eq total', 'notes']
df = pd.DataFrame(columns = cols)

In [157]:
df

,Last Name,First Name,openpowerlifting link,instagram,raw squat,raw bench,raw DL,raw total,eq squat,eq bench,eq DL,eq total,notes


In [158]:
def get_data(name):
    """
    scrape data from openpowerlifting
    input: firstnamelastname 
    """
    url = BASE_URL + name
    try:
        r = requests.get(url)
        df_list = pd.read_html(r.text) # this parses all the tables in webpages to a list
        df = df_list[0]
        df = df.dropna(axis=1, how='all')
        return df
    except ValueError as e:
        return pd.DataFrame()

In [159]:
get_data("fakename")

""


In [160]:
def build_df(first, last):
    """
    convert openpowerlifting format to kate spreadsheet format
    """
    result = {}
    df = pd.DataFrame(columns = cols)
    result['First Name'] = first
    result['Last Name'] = last
    
    # no special characters for openpl link lol
    first = re.sub(r'[^a-zA-Z0-9]', '', first)
    last = re.sub(r'[^a-zA-Z0-9]', '', last)
    name = first.lower() + last.lower()
    
    openpl_df = get_data(name)
    if not openpl_df.empty:
        raw = openpl_df[openpl_df['Equip'] == 'Raw'].drop(columns = ['Equip', 'Dots'])
        eq = openpl_df[openpl_df['Equip'] == 'Single'].drop(columns = ['Equip', 'Dots'])

        # lol this could be better but whatever
        result['openpowerlifting link'] = BASE_URL + name
        
        if not raw.empty:
            result['raw squat'] = raw['Squat'].iloc[0]
            result['raw bench'] = raw['Bench'].iloc[0]
            result['raw DL'] = raw['Deadlift'].iloc[0]
            result['raw total'] = raw['Total'].iloc[0]
        if not eq.empty:
            result['eq squat'] = eq['Squat'].iloc[0]
            result['eq bench'] = eq['Bench'].iloc[0]
            result['eq DL'] = eq['Deadlift'].iloc[0]
            result['eq total'] = eq['Total'].iloc[0]
    df = df.append(result, ignore_index=True)
    return df
    
    

In [161]:
meg = build_df("Megan", "Schroeder")

In [162]:
fake = build_df("fake", "name")

In [163]:
test = meg.append(fake)
test

,Last Name,First Name,openpowerlifting link,instagram,raw squat,raw bench,raw DL,raw total,eq squat,eq bench,eq DL,eq total,notes
0,Schroeder,Megan,https://www.openpowerlifting.org/u/meganschroeder,NaN,155,80,182.5,417.5,210,105,190.0,505.0,NaN
0,name,fake,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [164]:
# load roster
xls = pd.ExcelFile('cnats2023.xlsx')
roster_df = pd.read_excel(xls, 'Roster')
roster_df = roster_df[["Last Name", "First Name"]]
roster_df

,Last Name,First Name
0,Badeaux,Jessie
1,Ruff,Seth
2,Huerta,Vincent
3,Bowring,Ray
4,Belgard,Corbyn
...,...,...
268,Pulido,Jon
269,Riklan,Jonathan
270,Sanchez,Valeria
271,Serna,Nicholas


In [165]:
#populate our big sheet
for index, row in roster_df.iterrows():
    first = row['First Name']
    last = row['Last Name']
    df = df.append(build_df(first, last))
df

,Last Name,First Name,openpowerlifting link,instagram,raw squat,raw bench,raw DL,raw total,eq squat,eq bench,eq DL,eq total,notes
0,Badeaux,Jessie,https://www.openpowerlifting.org/u/jessiebadeaux,NaN,297.5,182.5,280,760,367.5,272.5,290,930,NaN
0,Ruff,Seth,https://www.openpowerlifting.org/u/sethruff,NaN,337.5,172.5,327.5,837.5,367.5,232.5,325.0,925.0,NaN
0,Huerta,Vincent,https://www.openpowerlifting.org/u/vincenthuerta,NaN,NaN,NaN,NaN,NaN,375,237.5,302.5,915,NaN
0,Bowring,Ray,https://www.openpowerlifting.org/u/raybowring,NaN,355,207.5,345,897.5,435,262.5,335,1030.0,NaN
0,Belgard,Corbyn,https://www.openpowerlifting.org/u/corbynbelgard,NaN,302.5,192.5,295.0,782.5,342.5,232.5,302.5,877.5,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,Pulido,Jon,https://www.openpowerlifting.org/u/jonpulido,NaN,NaN,NaN,NaN,NaN,195,110,202.5,507.5,NaN
0,Riklan,Jonathan,https://www.openpowerlifting.org/u/jonathanriklan,NaN,210,122.5,242.5,575,NaN,NaN,NaN,NaN,NaN
0,Sanchez,Valeria,https://www.openpowerlifting.org/u/valeriasanchez,NaN,142.5,80,150,372.5,NaN,NaN,NaN,NaN,NaN
0,Serna,Nicholas,https://www.openpowerlifting.org/u/nicholasserna,NaN,227.5,115,225,567.5,NaN,NaN,NaN,NaN,NaN


In [195]:
df.to_excel('bestlifts.xlsx', index = False)

In [166]:
gc = pygsheets.authorize(service_file='credentials.json')

In [167]:
# open sheet
sh = gc.open('Copy of Collegiate Nationals 2023')

#select the best lifters sheet
index = -1
count = 0
for s in sh:
    if s.title == "best lifts test":
        index = count
    count += 1
    
if index >= 0:
    wks = sh[index]
    wks.set_dataframe(df,(1,1), copy_index=False, nan="")